<a href="https://colab.research.google.com/github/yananma/5_programs_per_day/blob/master/02151.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 6.7 门控循环单元 ( GRU )

### 6.7.2 读取数据集

In [0]:
import numpy as np 
import torch 
from torch import nn, optim 
import torch.nn.functional as F 
import d2l 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
!mkdir ../../data 

In [5]:
!git clone https://github.com/ShusenTang/Dive-into-DL-PyTorch.git

Cloning into 'Dive-into-DL-PyTorch'...
remote: Enumerating objects: 1692, done.
remote: Total 1692 (delta 0), reused 0 (delta 0), pack-reused 1692
Receiving objects: 100% (1692/1692), 25.29 MiB | 14.15 MiB/s, done.
Resolving deltas: 100% (975/975), done.


In [0]:
!cp Dive-into-DL-PyTorch/data/jaychou_lyrics.txt.zip ../../data

In [0]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

### 6.7.3 从零开始实现

#### 1 初始化模型参数

In [8]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size 
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)), 
            _one((num_hiddens, num_hiddens)), 
            torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))

    W_xz, W_hz, b_z = _three()
    W_xr, W_hr, b_r = _three()
    W_xh, W_hh, b_h = _three()

    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q])

will use cuda


#### 2 定义模型

In [0]:
def init_gru_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [0]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params 
    H, = state 
    outputs = []
    for X in inputs:
        Z = torch.sigmoid(torch.matmul(X, W_xz) + torch.matmul(H, W_hz) + b_z)
        R = torch.sigmoid(torch.matmul(X, W_xr) + torch.matmul(H, W_hr) + b_r)
        H_tilda = torch.tanh(torch.matmul(X, W_xh) + R * torch.matmul(H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda 
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, )

#### 3 训练模型并创作歌词

In [0]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2 
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [13]:
d2l.train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens, vocab_size, device, corpus_indices, idx_to_char, 
            char_to_idx, False, num_epochs, num_steps, lr, clipping_theta, batch_size, pred_period, pred_len, 
            prefixes)

epoch 40, perplexity 151.758794, time 0.42 sec
 - 分开 我不不 我不不 我不不 我不不 我不不 我不不 我不不 我不不 我不不 我不不 我不不 我不不 我
 - 不分开 我想你我 你不我 你不了我 你不了我 你不了我 你不了我 你不了我 你不了我 你不了我 你不了我 
epoch 80, perplexity 32.830066, time 0.41 sec
 - 分开 我想要这样 我不要再想你 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 
 - 不分开 爱爱我 爱你 我想要这样 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 
epoch 120, perplexity 5.056568, time 0.43 sec
 - 分开我 一场好酒 你来一碗热粥 配色几斤 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍 
 - 不分开  没有你在我有多难熬多难熬  没有你烦 我有多烦恼  没有你在我有多难熬多难熬  没有你烦 我有多
epoch 160, perplexity 1.478603, time 0.42 sec
 - 分开 让我想大你 这对球 告诉真中 别人止中落不会一力危危 一个风 是果我外的菸以 一起好演日离 我来到
 - 不分开 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生活


### 6.7.4 简洁实现

In [14]:
lr = 1e-2 
gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens)
model = d2l.RNNModel(gru_layer, vocab_size).to(device)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device, corpus_indices, idx_to_char, char_to_idx, 
                num_epochs, num_steps, lr, clipping_theta, batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.019791, time 0.06 sec
 - 分开始的爱写在西元前 深埋在美索不达米亚平原 几十个世纪后出土发现 泥板上的字迹依然清晰可见 我给你的爱
 - 不分开始打呼 管家是一只会说法语举止优雅的猪 吸血前会念约翰福音做为弥补 拥有一双蓝色眼睛的凯萨琳公主 专
epoch 80, perplexity 1.031664, time 0.06 sec
 - 分开始想像 不想太多 我想一定是我听错弄错搞错 拜托 我想是你的脑袋有问题 随便说说 其实我早已经猜透看
 - 不分开始移动 回到当初爱你的时空 停格内容不忠 所有回忆对着我进攻 我的伤口被你拆封 誓言太沉重泪被纵容 
epoch 120, perplexity 1.010774, time 0.06 sec
 - 分开始想像 在回忆 的路上 时间变好慢 老街坊 小弄堂 是属于那年代白墙黑瓦的淡淡的忧伤 消失的 旧时光
 - 不分开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让
epoch 160, perplexity 1.013764, time 0.06 sec
 - 分开始它在空中停留 所有人看着我 抛物线进球 单手过人运球 篮下妙传出手 漂亮的假动作 帅呆了我 全场盯
 - 不分开始移动 回到当初爱你的时空 停格内容不忠 所有回忆对着我进攻 我的伤口被你拆封 誓言太沉重泪被纵容 
